<a href="https://colab.research.google.com/github/Rish-23072005/Mockmate/blob/main/Speech_recog_part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
%pip install ipywidgets
%pip install pyaudio
%pip install vosk
%pip install transformers
%pip install torch

  Using cached PyAudio-0.2.14.tar.gz (47 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for pyaudio (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pyaudio
Failed to build pyaudio
ERROR: Could not build wheels for pyaudio, which is required to install pyproject.toml-based projects
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 13.8 MB/s eta 0:00:00
  Created wheel for srt: filename=srt-3.5.3-py3-none-any.whl size=22428 sha256=5ddc70fa98016bdad56a32e6ae1d64f05a93bd47da7b0392da4f87bc80ac58af
  Stored in directory: /root/.cache/pip/wheels/d7/31/a

In [22]:
%pip install sounddevice
%pip install soundfile

In [24]:
!apt-get install -y portaudio19-dev
!pip install sounddevice

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libportaudio2 libportaudiocpp0
Suggested packages:
  portaudio19-doc
The following NEW packages will be installed:
  libportaudio2 libportaudiocpp0 portaudio19-dev
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 188 kB of archives.
After this operation, 927 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudio2 amd64 19.6.0-1.1 [65.3 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libportaudiocpp0 amd64 19.6.0-1.1 [16.1 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 portaudio19-dev amd64 19.6.0-1.1 [106 kB]
Fetched 188 kB in 1s (354 kB/s)
Selecting previously unselected package libportaudio2:amd64.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../libportaudio2_19.6.0-1.

In [ ]:
from flask import Flask, render_template, request, jsonify
from threading import Thread
import queue
import sounddevice as sd
import numpy as np
import subprocess
import json
from vosk import Model, KaldiRecognizer
import cv2
import base64

app = Flask(__name__)

# Queue for messages and recordings
messages = queue.Queue()
recordings = queue.Queue()

# Audio recording parameters
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 20
AUDIO_FORMAT = np.int16
SAMPLE_SIZE = 2

# Initialize Vosk model
model = Model(model_name="vosk-model-en-us-0.22")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)

def record_microphone():
    def callback(indata, frames, time, status):
        if status:
            print(status)
        recordings.put(indata.copy())

    with sd.InputStream(samplerate=FRAME_RATE, channels=CHANNELS, callback=callback, dtype=AUDIO_FORMAT):
        while not messages.empty():
            sd.sleep(1000)

def speech_recognition():
    while not messages.empty():
        frames = []
        while not recordings.empty():
            frames.append(recordings.get())
        if frames:
            frames = np.concatenate(frames, axis=0)
            rec.AcceptWaveform(frames)
            result = rec.Result()
            text = json.loads(result)["text"]
            return text

def capture_video():
    cap = cv2.VideoCapture(0)
    while not messages.empty():
        ret, frame = cap.read()
        if ret:
            _, jpeg = cv2.imencode('.jpg', frame)
            frame_data = base64.b64encode(jpeg.tobytes()).decode('utf-8')
            yield frame_data
        cv2.waitKey(1)
    cap.release()

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/start_recording', methods=['POST'])
def start_recording():
    messages.put(True)
    Thread(target=record_microphone).start()
    return jsonify({"message": "Recording started"})

@app.route('/stop_recording', methods=['POST'])
def stop_recording():
    if not messages.empty():
        messages.get()
    return jsonify({"message": "Recording stopped"})

@app.route('/get_transcription', methods=['GET'])
def get_transcription():
    transcription = speech_recognition()
    return jsonify({"transcription": transcription})

@app.route('/video_feed')
def video_feed():
    def generate():
        for frame in capture_video():
            yield f'data:image/jpeg;base64,{frame}\n'
    return app.response_class(generate(), mimetype='multipart/x-mixed-replace; boundary=frame')

if __name__ == '__main__':
    app.run(debug=True)


vosk-model-en-us-0.22.zip: 100%|██████████| 1.78G/1.78G [02:09<00:00, 14.8MB/s]


 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
